# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle below. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.


<img src="lifecycle.png" width="500">
<center><h4>Figure 1. Data Science Lifecycle</h4></center>

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [1]:
import urllib.request
quote_page = 'https://www.sfu.ca/computing/people/faculty.html'
page = urllib.request.urlopen(quote_page)
web_content = page.read()


f = open('csfaculty.html', 'wb')
f.write(web_content)
f.close

<function BufferedWriter.close>

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [2]:
# write your code
from bs4 import BeautifulSoup
import pandas as pd

html_soup = BeautifulSoup(open('csfaculty.html'), "html.parser")

profile_containers = html_soup.find_all(lambda tag: tag.name == 'div' and  tag.get('class') == ['text'])


names = []
ranks = []
areas = []
profiles =[]
homepages = []
index = 0

for each_profile in profile_containers:

    name_rank = each_profile.h4.text.split(", ")
    name = name_rank[0]
    rank= name_rank[1].rstrip()
   

    p_profiles = each_profile.find_all('p')
    
    # special case (Last profile)
    if index == 64:
        names.append(name)
        ranks.append(rank)
        areas.append('-')
        profiles.append('-')
        homepages.append('-')
        break
    
    # Special case
    if name =='Fred Popowich':
        area = rank.split('Area: ')[1]
        rank= name_rank[1].split('\n\n')[0]
    else:
        area = p_profiles[0].text.split('Area: ')[-1].rstrip()

    # to find index of p_profiles with href tag
    for i in range(len(p_profiles)):
        if p_profiles[i].select('a'):
            index_href = i    
            
    href_list = p_profiles[index_href].select('a')

    # assign missing homepage to be '-'
    if len(href_list) >1:
        profile = href_list[0].attrs['href']
        if href_list[1].has_attr('href'):
            homepage = href_list[1].attrs['href']
        else:
            homepage = '-'
    else:
        profile = href_list[0].attrs['href']
        homepage = '-'
        
    index += 1

    
    names.append(name)
    ranks.append(rank)
    areas.append(area)
    profiles.append(profile)
    homepages.append(homepage)

df = pd.DataFrame({'name': names,
                    'rank': ranks,
                    'area': areas,
                    'profile': profiles,
                    'homepage': homepages})

df.to_csv('faculty_table.csv', encoding='utf-8',index=False)
df

,name,rank,area,profile,homepage
0,Yagiz Aksoy,Assistant Professor,"Computational photography, computer graphics, ...",/computing/people/faculty/yagizaksoy.html,http://people.inf.ethz.ch/aksoyy/
1,Brad Bart,SENIOR LECTURER \nAssociate Director,Instruction,/computing/people/faculty/bradleybart.html,http://www.cs.sfu.ca/~bbart/
2,Angel Chang,Assistant Professor,"Area: Natural Language Processing, Artificial ...",/computing/people/faculty/angelchang.html,-
3,SHEELAGH CARPENDALE,PROFESSOR,Area: Information Visualization,/computing/people/faculty/sheelaghcarpendale.html,-
4,LEONID CHINDELEVITCH,ASSOCIATE PROFESSOR,Computational Biology,http://www.sfu.ca/computing/people/faculty/leo...,http://www.sfu.ca/~leonid
...,...,...,...,...,...
60,Richard Vaughan,Professor,Robotics and Autonomous Systems,/computing/people/faculty/richardvaughan.html,http://www.cs.sfu.ca/~vaughan/
61,Jiannan Wang,Assistant Professor,Database Systems and Big Data,/computing/people/faculty/jiannanwang.html,https://www.cs.sfu.ca/~jnwang/
62,Cynthia Xie,Senior Lecturer,Mandarin Chinese Instruction for the Dual Degr...,/computing/people/faculty/cynthiaxie.html,-
63,Richard (Hao) Zhang,Professor,Computer Graphics; Geometric Modelling,/computing/people/faculty/richardzhang.html,http://www.cs.sfu.ca/~haoz/


### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What intersting findings can you get from these visualizations? 

In [3]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

NumExpr defaulting to 8 threads.


GridBox(id='1191', ...)

Below are some examples:

**Finding 1:** Professor# (29) is more than 3x larger than Associate Professor# (9). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** Homepage has 20.3% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are intersted in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2020+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2020+23-1990 = 53. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 64 profile pages and save each page as a text file. 

In [4]:
# Write your code
import requests
import re
url = 'https://www.sfu.ca/computing/people/faculty.html'

# Connect to the URL
response = requests.get(url)

# Parse HTML and save to BeautifulSoup object
soup = BeautifulSoup(response.text, "html.parser")
profile_containers = html_soup.find_all(lambda tag: tag.name == 'div' and  tag.get('class') == ['text'])

index = 0
for each_profile in profile_containers:
    p_profiles = each_profile.find_all('p')#,text=re.compile(r"^\S"))
    
    # ignore last profile (the profile without homepage url)
    if index == 64:
        break

    # to find index of p_profiles with href tag
    for i in range(len(p_profiles)):
        if p_profiles[i].select('a'):
            index_href = i
            
    profile_page = p_profiles[index_href].select('a')[0].attrs['href']
    index += 1

    sfu_url ='www.sfu.ca'
    print(profile_page)
    if sfu_url in profile_page:
        quote_page = profile_page
    else:
        quote_page = "https://"+sfu_url+profile_page
    
#     print(quote_page)
    print(index, " \n")
    
    page = urllib.request.urlopen(quote_page)
    web_content = page.read()
    each_profile_saved = "profile_"+str(index)+".html"
    f = open(each_profile_saved, 'wb')
    f.write(web_content)
    f.close

/computing/people/faculty/yagizaksoy.html
1  

/computing/people/faculty/bradleybart.html
2  

/computing/people/faculty/angelchang.html
3  

/computing/people/faculty/sheelaghcarpendale.html
4  

http://www.sfu.ca/computing/people/faculty/leonidchindelevitch.html
5  

/computing/people/faculty/dianacukierman.html
6  

/computing/people/faculty/jamesdelgrande.html
7  

/computing/people/faculty/tobydonaldson.html
8  

/computing/people/faculty/johnedgar.html
9  

/computing/people/faculty/martinester.html
10  

/computing/people/faculty/brianfraser.html
11  

/computing/people/faculty/yasutakafurukawa1.html
12  

/computing/people/faculty/uweglasser.html
13  

/computing/people/faculty/mohamedhefeeda.html
14  

/computing/people/faculty/harindersinghkhangura.html
15  

/computing/people/faculty/annelavergne.html
16  

/computing/people/faculty/maxlibbrecht11.html
17  

/computing/people/faculty/jiangchuanliu.html
18  

/computing/people/faculty/davidmitchell.html
19  

/computing/peopl

### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [5]:
# write your code here

names = []
grad_years = []
for i in range(1,65):
    filename = 'profile_'+str(i)+".html"
    html_soup = BeautifulSoup(open(filename,encoding="utf8"), "html.parser")

    profile_containers = html_soup.find_all('div',class_='text parbase section')
    name = html_soup.find('div',class_='title section').text
#     print(name)

    index_education = 3
    # to find index of text.parbase.section with Education
    for i in range(len(profile_containers)):
        try:
            # One special case
            if name == 'Hang Ma' :
                index_education = 3
                break
            if profile_containers[i].h2.text=='Education':
                index_education = i
                break
        except (AttributeError, KeyError):
            pass    

    # remove non-string from list and split them by lines
    list_education =profile_containers[index_education].text.strip().splitlines()
    # remove empty string from list
    list_education = list(filter(None,list_education))#.splitlines()#[1]
    #print(list_education)
    
    # get latest education and grad year
    latest_grad_year = list_education[1][-4:]
    
    # assume missing year equal to 2019
    if not latest_grad_year.isnumeric():
        latest_grad_year = 2019
    else:
        latest_grad_year = int(latest_grad_year)
    names.append(name)
    grad_years.append(latest_grad_year)
    
df = pd.DataFrame({'name': names,
                    'gradyear': grad_years})

df.to_csv("faculty_grad_year.csv", encoding='utf-8',index=False)
df

,name,gradyear
0,Yagiz Aksoy,2019
1,Bradley Bart,2000
2,ANGEL CHANG,2015
3,SHEELAGH CARPENDALE,1999
4,Leonid Chindelevitch,2010
...,...,...
59,Oliver Schulte,1997
60,Richard T. Vaughan,1998
61,Jiannan Wang,2013
62,Cynthia Xie,2019


### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [6]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
# print(df)
df["age"] = 2020+23-df["gradyear"]

plot(df, "age")

Tabs(id='1574', ...)

## Submission

Complete the code in this [notebook](https://github.com/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1.ipynb), and submit it to the CourSys activity `Assignment 1`.

<hr/>
## <span style="color:red">Bonus: Contribute to dataprep</span>

**Total Bonus: 0.2 + 0.3 = 0.5**

1. If you create an issue (i.e., report a bug or request a feature) at [link1](https://github.com/sfu-db/dataprep/issues) or [link2](https://github.com/sfu-db/DataConnectorConfigs/issues), you will get **0.2** bonus points.

2.  Creating more issues will *not* give you more bonus points, but you are encouraged to do so for learning more.

2. If you send a pull request (i.e., fix a bug or implement a feature or add data connector for a new website) at [link1](https://github.com/sfu-db/dataprep/pulls) or [link2](https://github.com/sfu-db/DataConnectorConfigs/pulls) and the pull request gets merged into the repo, you will get **0.3** bonus points. 

4.  Sending more pull requests will *not* give you more bonus points, but you are encouraged to do so for learning more.

5. These bonus points will be directly added to your final grade.






**How to Submit**
* Submit github link(s) to the CourSys activity `Bonus`.
* Due on `March 15, 2020`

If you love dataprep, please support it by simply clicking the **<span style="color:red">Star</span>** on [Github](https://github.com/sfu-db/dataprep).
<img src="dataprep-star.png" width="1000">